In [52]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import string

from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
%matplotlib inline

import tensorflow_hub as hub

In [6]:
df = pd.read_csv('/content/drive/MyDrive/test.csv')
df.head()

lablel                                            Column1  \
0  __label__2                                           Great CD   
1  __label__2   One of the best game music soundtracks - for ...   
2  __label__1                   Batteries died within a year ...   
3  __label__2              works fine, but Maha Energy is better   
4  __label__2                       Great for the non-audiophile   

                                             Column2  
0   My lovely Pat has one of the GREAT voices of ...  
1   Despite the fact that I have only played a sm...  
2   I bought this charger in Jul 2003 and it work...  
3   Check out Maha Energy's website. Their Powere...  
4   Reviewed quite a bit of the combo players and...

In [7]:
df.describe()

lablel        Column1 Column2
count       400000         400000  399840
unique           2         324400  398465
top     __label__2   Disappointed       (
freq        200000           1048     197

In [8]:
df.fillna(value='', inplace=True)

In [9]:
df.isna().sum()

lablel     0
Column1    0
Column2    0
dtype: int64

## Label encoding

In [13]:
df['lablel'] = df['lablel'].astype('category')

# Assigning numerical values and storing it in another columns
df['lab'] = df['lablel'].cat.codes

# Create an instance of One-hot-encoder
enc = OneHotEncoder()

enc_df = pd.DataFrame(enc.fit_transform(
    df[['lab']]).toarray())

New_df = df.join(enc_df)

print(New_df)



            lablel                                            Column1  \
0       __label__2                                           Great CD   
1       __label__2   One of the best game music soundtracks - for ...   
2       __label__1                   Batteries died within a year ...   
3       __label__2              works fine, but Maha Energy is better   
4       __label__2                       Great for the non-audiophile   
...            ...                                                ...   
399995  __label__1                         Unbelievable- In a Bad Way   
399996  __label__1                    Almost Great, Until it Broke...   
399997  __label__1                                   Disappointed !!!   
399998  __label__2                            Classic Jessica Mitford   
399999  __label__1                        Comedy Scene, and Not Heard   

                                                  Column2  lab    0    1  
0        My lovely Pat has one of the GREAT voic

In [14]:
df.head(20)

lablel                                            Column1  \
0   __label__2                                           Great CD   
1   __label__2   One of the best game music soundtracks - for ...   
2   __label__1                   Batteries died within a year ...   
3   __label__2              works fine, but Maha Energy is better   
4   __label__2                       Great for the non-audiophile   
5   __label__1              DVD Player crapped out after one year   
6   __label__1                                     Incorrect Disc   
7   __label__1                           DVD menu select problems   
8   __label__2              Unique Weird Orientalia from the 1930   
9   __label__1                            Not an "ultimate guide"   
10  __label__2                   Great book for travelling Europe   
11  __label__1                                               Not!   
12  __label__1                                    A complete Bust   
13  __label__2                           TRULY MADE A DIFFERENCE!   
14  __label__1                    didn't run off of USB bus power   
15  __label__1                                         Don't buy!   
16  __label__2             Simple, Durable, Fun game for all ages   
17  __label__2                  Review of Kelly Club for Toddlers   
18  __label__2                          SOY UN APASIONADO DEL BOX   
19  __label__2   Some of the best fiddle playing I have heard ...   

                                              Column2  lab  
0    My lovely Pat has one of the GREAT voices of ...    1  
1    Despite the fact that I have only played a sm...    1  
2    I bought this charger in Jul 2003 and it work...    0  
3    Check out Maha Energy's website. Their Powere...    1  
4    Reviewed quite a bit of the combo players and...    1  
5    I also began having the incorrect disc proble...    0  
6    I love the style of this, but after a couple ...    0  
7    I cannot scroll through a DVD menu that is se...    0  
8    Exotic tales of the Orient from the 1930's. "...    1  
9    Firstly,I enjoyed the format and tone of the ...    0  
10   I currently live in Europe, and this is the b...    1  
11   If you want to listen to El Duke , then it is...    0  
12   This game requires quicktime 5.0 to work...if...    0  
13   I have been using this product for a couple y...    1  
14   Was hoping that this drive would run off of b...    0  
15   First of all, the company took my money and s...    0  
16   This is an AWESOME game! Almost everyone know...    1  
17   For the price of 7.99, this PC game is WELL w...    1  
18   Y ESTE LIBRO ESTÁ ESPLÉNDIDO !Lo disfrutas, l...    1  
19   This is an excellent album with some great fi...    1

## Taking required column

In [16]:
df = df[['Column2','lab']]
df.head(20)

Column2  lab
0    My lovely Pat has one of the GREAT voices of ...    1
1    Despite the fact that I have only played a sm...    1
2    I bought this charger in Jul 2003 and it work...    0
3    Check out Maha Energy's website. Their Powere...    1
4    Reviewed quite a bit of the combo players and...    1
5    I also began having the incorrect disc proble...    0
6    I love the style of this, but after a couple ...    0
7    I cannot scroll through a DVD menu that is se...    0
8    Exotic tales of the Orient from the 1930's. "...    1
9    Firstly,I enjoyed the format and tone of the ...    0
10   I currently live in Europe, and this is the b...    1
11   If you want to listen to El Duke , then it is...    0
12   This game requires quicktime 5.0 to work...if...    0
13   I have been using this product for a couple y...    1
14   Was hoping that this drive would run off of b...    0
15   First of all, the company took my money and s...    0
16   This is an AWESOME game! Almost everyone know...    1
17   For the price of 7.99, this PC game is WELL w...    1
18   Y ESTE LIBRO ESTÁ ESPLÉNDIDO !Lo disfrutas, l...    1
19   This is an excellent album with some great fi...    1

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   lablel   400000 non-null  category
 1   Column1  400000 non-null  object  
 2   Column2  400000 non-null  object  
 3   lab      400000 non-null  int8    
dtypes: category(1), int8(1), object(2)
memory usage: 6.9+ MB


In [17]:
df['Column2']=df['Column2'].str.lower()
df.head()

<ipython-input-17-cb21a53ca2d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Column2']=df['Column2'].str.lower()


Column2  lab
0   my lovely pat has one of the great voices of ...    1
1   despite the fact that i have only played a sm...    1
2   i bought this charger in jul 2003 and it work...    0
3   check out maha energy's website. their powere...    1
4   reviewed quite a bit of the combo players and...    1

## Removing stopwords

In [18]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
from nltk.corpus import stopwords

In [20]:
",".join(stopwords.words('english'))

"i,me,my,myself,we,our,ours,ourselves,you,you're,you've,you'll,you'd,your,yours,yourself,yourselves,he,him,his,himself,she,she's,her,hers,herself,it,it's,its,itself,they,them,their,theirs,themselves,what,which,who,whom,this,that,that'll,these,those,am,is,are,was,were,be,been,being,have,has,had,having,do,does,did,doing,a,an,the,and,but,if,or,because,as,until,while,of,at,by,for,with,about,against,between,into,through,during,before,after,above,below,to,from,up,down,in,out,on,off,over,under,again,further,then,once,here,there,when,where,why,how,all,any,both,each,few,more,most,other,some,such,no,nor,not,only,own,same,so,than,too,very,s,t,can,will,just,don,don't,should,should've,now,d,ll,m,o,re,ve,y,ain,aren,aren't,couldn,couldn't,didn,didn't,doesn,doesn't,hadn,hadn't,hasn,hasn't,haven,haven't,isn,isn't,ma,mightn,mightn't,mustn,mustn't,needn,needn't,shan,shan't,shouldn,shouldn't,wasn,wasn't,weren,weren't,won,won't,wouldn,wouldn't"

In [21]:
stop_words = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop_words.update(punctuation)

def remove_stop(x):
  return ",".join([word for word in str(x).split() if word not in stop_words])
df['Column2'].apply(lambda x : remove_stop(x))

0         lovely,pat,one,great,voices,generation.,listen...
1         despite,fact,played,small,portion,game,,music,...
2         bought,charger,jul,2003,worked,ok,while.,desig...
3         check,maha,energy's,website.,powerex,mh-c204f,...
4         reviewed,quite,bit,combo,players,hesitant,due,...
                                ...                        
399995    bought,thomas,son,huge,thomas,fan,,huge,set,ro...
399996    son,recieved,birthday,gift,2,months,ago.,loved...
399997    bought,toy,son,loves,"thomas",toys.,need,one,b...
399998    compilation,wide,range,mitford's,articles,,bes...
399999    dvd,disappointment,get,hoping,see,substantial,...
Name: Column2, Length: 400000, dtype: object

## Cleaning the text

In [22]:

def strip_html(Column2):
    soup = BeautifulSoup(Column2, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(Column2):
    return re.sub('\[[^]]*\]', '', Column2)
# Removing URL's++
def remove_between_square_brackets(Column2):
    return re.sub(r'http\S+', '', Column2)



## Spliting Words

In [23]:
words = []
for i in df.Column2.values:
    words.append(i.split())
words[:5]

[['my',
  'lovely',
  'pat',
  'has',
  'one',
  'of',
  'the',
  'great',
  'voices',
  'of',
  'her',
  'generation.',
  'i',
  'have',
  'listened',
  'to',
  'this',
  'cd',
  'for',
  'years',
  'and',
  'i',
  'still',
  'love',
  'it.',
  'when',
  "i'm",
  'in',
  'a',
  'good',
  'mood',
  'it',
  'makes',
  'me',
  'feel',
  'better.',
  'a',
  'bad',
  'mood',
  'just',
  'evaporates',
  'like',
  'sugar',
  'in',
  'the',
  'rain.',
  'this',
  'cd',
  'just',
  'oozes',
  'life.',
  'vocals',
  'are',
  'jusat',
  'stuunning',
  'and',
  'lyrics',
  'just',
  'kill.',
  'one',
  'of',
  "life's",
  'hidden',
  'gems.',
  'this',
  'is',
  'a',
  'desert',
  'isle',
  'cd',
  'in',
  'my',
  'book.',
  'why',
  'she',
  'never',
  'made',
  'it',
  'big',
  'is',
  'just',
  'beyond',
  'me.',
  'everytime',
  'i',
  'play',
  'this,',
  'no',
  'matter',
  'black,',
  'white,',
  'young,',
  'old,',
  'male,',
  'female',
  'everybody',
  'says',
  'one',
  'thing',
  '"wh

## Tokenizing->numbers

In [25]:
max_features = 35000
maxlen = 200

In [26]:
tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, split=' ')

In [27]:
tokenizer = Tokenizer(num_words=35000)
tokenizer.fit_on_texts(words)
tokenized_ind = tokenizer.texts_to_sequences(words)
tokenized_ind

[[20,
  2043,
  5871,
  39,
  25,
  6,
  1,
  45,
  2400,
  6,
  75,
  8891,
  3,
  19,
  892,
  5,
  7,
  122,
  11,
  157,
  2,
  3,
  114,
  85,
  63,
  43,
  102,
  10,
  4,
  41,
  2555,
  9,
  176,
  57,
  201,
  514,
  4,
  168,
  2555,
  33,
  30,
  3222,
  10,
  1,
  13757,
  7,
  122,
  33,
  28755,
  587,
  1316,
  21,
  2,
  755,
  33,
  22089,
  25,
  6,
  7715,
  2825,
  16955,
  7,
  8,
  4,
  5934,
  22417,
  122,
  10,
  20,
  127,
  187,
  87,
  109,
  129,
  9,
  220,
  8,
  33,
  837,
  270,
  2866,
  3,
  235,
  588,
  53,
  632,
  6296,
  6030,
  6357,
  1941,
  21100,
  1389,
  2696,
  499,
  25,
  172,
  7361,
  13,
  12,
  1007],
 [966,
  1,
  373,
  12,
  3,
  19,
  51,
  481,
  4,
  257,
  2726,
  6,
  1,
  1380,
  1,
  152,
  3,
  305,
  14087,
  1,
  1569,
  5,
  29379,
  6031,
  86,
  13,
  45,
  22,
  17666,
  1727,
  57,
  5,
  347,
  1,
  6900,
  2,
  9,
  2478,
  25,
  6,
  20,
  312,
  2317,
  59,
  8,
  38,
  1254,
  1049,
  6,
  1671,
  18677,
  2,


model implementation

In [28]:
sentlen=20
emdoc=pad_sequences(tokenized_ind,padding='post',maxlen=sentlen)
print(emdoc)

[[2866    3  235 ...   13   12 1007]
 [   1 1244   86 ...  513  160   63]
 [  22   93   33 ...  110 3977 4151]
 ...
 [   1    1 3852 ... 1400   11 4654]
 [ 184   22   75 ... 4346    5  361]
 [   7    2   42 ...    5  181 2265]]


In [31]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1 # 286300+1
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sentlen))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid',)) # for classification problem
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            30763680  
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 30,820,181
Trainable params: 30,820,181
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
X=np.array(emdoc)
y=df['lab']
y=np.array(y)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Training the Model

In [39]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=5,batch_size=64)


Epoch 1/5
4375/4375 [==============================] - 66s 15ms/step - loss: 0.3294 - accuracy: 0.8543 - val_loss: 0.3927 - val_accuracy: 0.8193
Epoch 2/5
4375/4375 [==============================] - 63s 14ms/step - loss: 0.2766 - accuracy: 0.8805 - val_loss: 0.4236 - val_accuracy: 0.8166
Epoch 3/5
4375/4375 [==============================] - 64s 15ms/step - loss: 0.2250 - accuracy: 0.9055 - val_loss: 0.4945 - val_accuracy: 0.8084
Epoch 4/5
4375/4375 [==============================] - 62s 14ms/step - loss: 0.1772 - accuracy: 0.9268 - val_loss: 0.5706 - val_accuracy: 0.8037
Epoch 5/5
4375/4375 [==============================] - 62s 14ms/step - loss: 0.1354 - accuracy: 0.9454 - val_loss: 0.6709 - val_accuracy: 0.7919


Model Evaluation

In [56]:
print("Accuracy of the model on Training Data is - " , model.evaluate(x_train,y_train)[1]*100)
print("Accuracy of the model on Testing Data is - " , model.evaluate(x_test,y_test)[1]*100)

8750/8750 [==============================] - 32s 4ms/step - loss: 0.0953 - accuracy: 0.9679
Accuracy of the model on Training Data is -  96.79428339004517
3750/3750 [==============================] - 13s 4ms/step - loss: 0.6709 - accuracy: 0.7919
Accuracy of the model on Testing Data is -  79.19333577156067


In [55]:
def set_threshold(predictions, threshold):
  """Sets a threshold for the predicted values and converts them to 0 or 1."""
  binary_predictions = np.where(predictions >= threshold, 1, 0)
  return binary_predictions

predictions = model.predict(x_test)
binary_predictions = set_threshold(predictions, 0.5)
accuracy = accuracy_score(y_test, binary_predictions)
precision = precision_score(y_test, binary_predictions)
recall = recall_score(y_test, binary_predictions)
f1 = f1_score(y_test, binary_predictions)

print(" accuracy:", accuracy)
print(" precision:",precision)
print(" recall:", recall)
print(" f1:", f1)




3750/3750 [==============================] - 8s 2ms/step
 accuracy: 0.7919333333333334
 precision: 0.7869056795630853
 recall: 0.8002668668167793
 f1: 0.793530034400635
